# Polar Flow 
Use the Polar Flow API to get the data from your Polar device.

    Website: https://www.polar.com/accesslink-api/#introduction

    Github https://github.com/polarofficial
    
    Github: https://github.com/polarofficial/accesslink-example-python
    
    Swagger	https://www.polar.com/accesslink-api/swagger.yaml

    

In [1]:
import requests
headers = {
  'Accept': 'application/json',
  'Authorization': ''
}

r = requests.post('https://www.polaraccesslink.com/v3/users/42290156/activity-transactions', params={

}, headers = headers)

print(r)

<Response [401]>


In [3]:
import requests
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer {access-token}'
}

r = requests.post('https://www.polaraccesslink.com/v3/users/{user-id}/activity-transactions', params={

}, headers = headers)

print(r.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
#!/usr/bin/env python

import endpoints
from .oauth2 import OAuth2Client

AUTHORIZATION_URL = "https://flow.polar.com/oauth2/authorization"
ACCESS_TOKEN_URL = "https://polarremote.com/v2/oauth2/token"
ACCESSLINK_URL = "https://www.polaraccesslink.com/v3"


class AccessLink(object):
    """Wrapper class for Polar Open AccessLink API v3"""

    def __init__(self, client_id, client_secret, redirect_url=None):
        if not client_id or not client_secret:
            raise ValueError("Client id and secret must be provided.")

        self.oauth = OAuth2Client(url=ACCESSLINK_URL,
                                  authorization_url=AUTHORIZATION_URL,
                                  access_token_url=ACCESS_TOKEN_URL,
                                  redirect_url=redirect_url,
                                  client_id=client_id,
                                  client_secret=client_secret)

        self.users = endpoints.Users(oauth=self.oauth)
        self.pull_notifications = endpoints.PullNotifications(oauth=self.oauth)
        self.training_data = endpoints.TrainingData(oauth=self.oauth)
        self.physical_info = endpoints.PhysicalInfo(oauth=self.oauth)
        self.daily_activity = endpoints.DailyActivity(oauth=self.oauth)

    @property
    def authorization_url(self):
        """Get the authorization url for the client"""
        return self.oauth.get_authorization_url()

    def get_access_token(self, authorization_code):
        """Request access token for a user.
        :param authorization_code: authorization code received from authorization endpoint.
        """
        return self.oauth.get_access_token(authorization_code)

ModuleNotFoundError: No module named 'endpoints'

In [6]:
!pip install utils

In [9]:
!pip install accesslink

ERROR: Could not find a version that satisfies the requirement accesslink (from versions: none)
ERROR: No matching distribution found for accesslink


In [8]:
#!/usr/bin/env python
from __future__ import print_function

import requests
from flask import Flask, request, redirect

import utils
#from utils import load_config, save_config
from accesslink import AccessLink


CALLBACK_PORT = 5000
CALLBACK_ENDPOINT = "/oauth2_callback"

CONFIG_FILENAME = "config.yml"

REDIRECT_URL = "http://localhost:{}{}".format(CALLBACK_PORT, CALLBACK_ENDPOINT)

config = load_config(CONFIG_FILENAME)

accesslink = AccessLink(client_id=config['c6a9a0b9-fa1c-44d5-b1ca-4bef736236fc'],
                        client_secret=config['f70c1621-952e-4285-88b6-bc2053199368'],
                        redirect_url=REDIRECT_URL)


app = Flask(__name__)


@app.route("/")
def authorize():
    return redirect(accesslink.authorization_url)


@app.route(CALLBACK_ENDPOINT)
def callback():
    """Callback for OAuth2 authorization request
    Saves the user's id and access token to a file.
    """

    #
    # Get authorization from the callback request parameters
    #
    authorization_code = request.args.get("code")

    #
    # Get an access token for the user using the authorization code.
    #
    # The authorization code is only valid for 10 minutes, so the access token
    # should be fetched immediately after the authorization step.
    #
    token_response = accesslink.get_access_token(authorization_code)

    #
    # Save the user's id and access token to the configuration file.
    #
    config["user_id"] = token_response["x_user_id"]
    config["access_token"] = token_response["access_token"]
    save_config(config, CONFIG_FILENAME)

    #
    # Register the user as a user of the application.
    # This must be done before the user's data can be accessed through AccessLink.
    #
    try:
        accesslink.users.register(access_token=config["access_token"])
    except requests.exceptions.HTTPError as err:
        # Error 409 Conflict means that the user has already been registered for this client.
        # That error can be ignored in this example.
        if err.response.status_code != 409:
            raise err

    shutdown()
    return "Client authorized! You can now close this page."


def shutdown():
    shutdown_func = request.environ.get('werkzeug.server.shutdown')
    if shutdown_func is not None:
        shutdown_func()


def main():
    print("Navigate to http://localhost:{port}/ for authorization.\n".format(port=CALLBACK_PORT))
    app.run(host='localhost', port=CALLBACK_PORT)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'accesslink'